West Wind (Joins) (Dataframe version)
Data sources:

https://gitlab.com/opstar/share20/-/raw/master/westwcustomer.txt?inline=false

https://gitlab.com/opstar/share20/-/raw/master/westwproduct.txt?inline=false

https://gitlab.com/opstar/share20/-/raw/master/westworder.txt?inline=false

In [ ]:
rawcust = spark.read.format('csv').option('delimiter','|').load('/FileStore/sample/westwcustomer.txt')
rawprod = spark.read.format('csv').option('delimiter','|').load('/FileStore/sample/westwproduct.txt')
rawordr = spark.read.format('csv').option('delimiter','|').load('/FileStore/sample/westworder.txt')

cust = rawcust.toDF('custnum','companyname','city','postcode','country')
prod = rawprod.toDF('productnum','productname','categoryname','unitprice','unitstock')
ordr = rawordr.toDF('ordernum','custnum','orderdate','shipname','shipaddress','shipcity','shipregion','shippostcode','shipcountry','productnum','unitprice','quantity','discount')

In [3]:
from pyspark.sql.functions import count

In [ ]:
## Q3 Count the Seafood orders - distinct() version
prod.where(prod.categoryname == 'Seafood')\
    .join(ordr,'productnum')\
    .select('ordernum').distinct().count()

In [ ]:
291

In [ ]:
## Q3 Count the Seafood orders - groupBy countDistinct() version
from pyspark.sql.functions import countDistinct,count

prod.where(prod.categoryname == 'Seafood')\
    .join(ordr,'productnum')\
    .groupBy().agg(countDistinct('ordernum')).show()

In [ ]:
+---------------+
|count(ordernum)|
+---------------+
|            291|
+---------------+


In [ ]:
## Q3 Alternative solutions demonstratring the same answer with both inner and leftSemi
ordr.join(prod.where(prod.categoryname == 'Seafood'),'productnum','inner')\
    .agg(countDistinct('ordernum')).show()

ordr.join(prod.where(prod.categoryname == 'Seafood'),'productnum','leftSemi')\
    .agg(countDistinct('ordernum')).show()

In [ ]:
+---------------+
|count(ordernum)|
+---------------+
|            291|
+---------------+

+---------------+
|count(ordernum)|
+---------------+
|            291|
+---------------+


In [ ]:
## Q4 Customers who have not placed an order (anti-join)
cust.join(ordr,'custnum','leftAnti').show()

In [ ]:
+-------+--------------------+------+--------+-------+
|custnum|         companyname|  city|postcode|country|
+-------+--------------------+------+--------+-------+
|  PARIS|   Paris spécialités| Paris|   75012| France|
|  FISSA|FISSA Fabrica Int...|Madrid|   28034|  Spain|
+-------+--------------------+------+--------+-------+


In [ ]:
cust.join(ordr,cust.custnum == ordr.custnum,'LeftOuter').select(cust.custnum,cust.companyname,cust.city,cust.postcode,cust.country).where(ordr.ordernum.isNull()).show()

In [ ]:
+-------+--------------------+------+--------+-------+
|custnum|         companyname|  city|postcode|country|
+-------+--------------------+------+--------+-------+
|  PARIS|   Paris spécialités| Paris|   75012| France|
|  FISSA|FISSA Fabrica Int...|Madrid|   28034|  Spain|
+-------+--------------------+------+--------+-------+


In [ ]:
cust.join(ordr,cust.custnum == ordr.custnum,'LeftOuter').select(cust.custnum,cust.companyname,cust.city,cust.postcode,cust.country).where(ordr.ordernum.isNull()).show()

In [ ]:
+-------+--------------------+------+--------+-------+
|custnum|         companyname|  city|postcode|country|
+-------+--------------------+------+--------+-------+
|  PARIS|   Paris spécialités| Paris|   75012| France|
|  FISSA|FISSA Fabrica Int...|Madrid|   28034|  Spain|
+-------+--------------------+------+--------+-------+
